Notebook and data of Ruben Brabenec.

This information, data and code belong to Ruben Brabenec (Helmholtz Zentrum München). 

Disclosure to third parties is prohibited and requires consent.

This Python script represents a general approach to single-cell RNA-seq data analysis. Variations may occur depending on the specific format of the raw data files, the thresholds that need to be applied during quality control, the types of cells being analyzed, and the patient data or diseases involved. Adjustments may be required based on the specific experimental setup and the biological context of the study.


In [ ]:
#pip install sfaira

In [ ]:
#pip install anndata

In [ ]:
#pip install mygene

In [ ]:
#pip install scanpy

In [ ]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import anndata
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
import mygene
mg = mygene.MyGeneInfo()
import os
import sfaira

from gprofiler import gprofiler
from os import listdir
from os.path import isfile, join

import warnings
from rpy2.rinterface import RRuntimeWarning
from rpy2.robjects import pandas2ri

# Load R extension for running R code within the Python environment
%load_ext rpy2.ipython

# Suppress R warnings to avoid clutter during analysis
warnings.filterwarnings("ignore", category=RRuntimeWarning)

# Activate the pandas2ri interface for seamless conversion between pandas and R data frames
pandas2ri.activate()

# Configure pandas display settings for better output visibility
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Set verbosity for Scanpy operations to provide detailed logging during analysis
sc.settings.verbosity = 3

# Define a custom color map for visualizing gene expression values in heatmaps or other figures
colors2 = plt.cm.Reds(np.linspace(0, 1, 128))
colors3 = plt.cm.Greys_r(np.linspace(0.7, 0.8, 20))
colorsComb = np.vstack([colors3, colors2])
mymap = colors.LinearSegmentedColormap.from_list('my_colormap', colorsComb)

# Set default plotting parameters for Scanpy, ensuring high-quality vector-friendly outputs
sc.set_figure_params(vector_friendly=True, color_map='Reds',
                     dpi=200, transparent=True, fontsize=14)

In [ ]:
# Define the file path where all raw data files are located
path = ...'raw_data/'


# Curating sample

In [ ]:
# Read the h5ad file into an AnnData object
adata = sc.read_h5ad(file_path)

In [ ]:
# Define the paths for the obs and var CSV files
obs_csv_path = path + 'obs_data.csv'
var_csv_path = path + 'var_data.csv'

In [ ]:
# Read obs and var metadata from CSV files
obs_df = pd.read_csv(obs_csv_path, index_col=0)
var_df = pd.read_csv(var_csv_path, index_col=0)

In [ ]:
# Ensure that the indices of the obs and var DataFrames match the dimensions of adata
# This is crucial for correctly aligning metadata with the expression matrix
assert adata.shape[0] == obs_df.shape[0], "Number of cells in obs does not match the adata dimensions."
assert adata.shape[1] == var_df.shape[0], "Number of genes in var does not match the adata dimensions."

In [ ]:
# Assign the obs and var data to the adata object
adata.obs = obs_df
adata.var = var_df

In [ ]:
adata.strings_to_categoricals()

In [ ]:
adata.X

In [ ]:
adata.obs = adata.obs.astype(str)

In [ ]:
adata.X.shape

In [ ]:
adata.shape

# Preprocess sample

In [ ]:
# Basic quality control (QC) - filter cells and genes based on basic QC metrics
# Filter cells with fewer than 200 genes and filter genes expressed in fewer than 3 cells
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
# Calculate mitochondrial gene fraction
# Assuming mitochondrial genes are labeled with 'MT-' in gene names
adata.var['mt'] = adata.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts")

In [ ]:
# Filter cells based on QC metrics
# Remove cells with high mitochondrial gene content (>5%) and cells with extremely low or high counts
adata = adata[adata.obs['pct_counts_mt'] < 5, :]
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.n_genes_by_counts > 200, :]

In [ ]:
# Normalize data
# Normalize each cell by the total counts over all genes, so that every cell has the same total count
# Multiply the normalized values by a scaling factor of 10,000
sc.pp.normalize_total(adata, target_sum=1e4)

In [ ]:
# Logarithmic transformation
# Apply logarithm to each gene's expression to normalize the distribution
sc.pp.log1p(adata)

In [ ]:
# Identify highly variable genes (HVGs)
# These genes are used for downstream analysis to capture the most informative genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
# Retain only the highly variable genes for downstream analysis
#adata = adata[:, adata.var.highly_variable]

In [ ]:
# Regress out unwanted sources of variation
# Regress out total counts per cell and the percentage of mitochondrial genes to reduce noise
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

In [ ]:
# Scale the data
# Each gene is centered to mean 0 and scaled to unit variance for better downstream analysis
sc.pp.scale(adata, max_value=10)

# Principal Component Analysis (PCA)

In [ ]:
# Perform PCA (Principal Component Analysis) on the scaled data
sc.tl.pca(adata, svd_solver='arpack')

# Visualize the variance explained by each principal component
sc.pl.pca_variance_ratio(adata, log=True)

# Computing the neighborhood graph

In [ ]:
# Compute the neighborhood graph of cells
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

# UMAP

In [ ]:
# Compute UMAP (Uniform Manifold Approximation and Projection)
sc.tl.umap(adata)

In [ ]:
# Visualize the cells in UMAP space
sc.pl.umap(adata, color=['total_counts', 'pct_counts_mt', 'n_genes_by_counts'])

In [ ]:
sc.pl.umap(adata, color=["Gen1", "Gen2", "Gen3"])

# Annotation

In [ ]:
# Define the 'celltype' categorical variable based on existing categories
ref_cluster = pd.Categorical(adata.obs['celltype'], 
                             categories=['Celltype-1-A', 'Celltype-1-B', 'Celltype-2-A', 'Celltype-2-B'])

In [ ]:
# Consolidate subtypes into broader categories
ix = np.isin(ref_cluster, ['Celltype-1-A', 'Celltype-1-B'])
ref_cluster[ix] = 'Celltype-1'

In [ ]:
ix = np.isin(ref_cluster, ['Celltype-2-A', 'Celltype-2-B'])
ref_cluster[ix] = 'Celltype-2'

In [ ]:
# Update the 'celltype' column in the AnnData object with the new categories
adata.obs['celltype'] = pd.Categorical(ref_cluster, 
                                       categories=['Celltype-1', 'Celltype-2'])

In [ ]:
# Rename categories if necessary
adata.rename_categories('celltype', ['Type 1', 'Type 2'])

In [ ]:
# Visualize the new cell type assignments using UMAP
sc.pl.umap(adata, color=['celltype'])

In [ ]:
# Ensure all string variables are converted to categorical
adata.strings_to_categoricals()

In [ ]:
adata.obs['Malignant'] = 'NonMalignant'
adata.obs['celltype_2'] = 'Celltype'

for i in adata.obs.index:
    if adata.obs['celltype'][i] == 'Malignangt Cell':
            adata.obs['Malignant'][i] = 'Malignant'
            adata.obs['celltype_2'][i] = 'Malignangt Cell'
    else:
        adata.obs['Malignant'][i] = 'NonMalignant'
        adata.obs['celltype_2'][i] = adata.obs['celltype'][i]

In [ ]:
adata.strings_to_categoricals()

In [ ]:
adata.obs['Organ'] = #Organ
adata.obs['Organ_Specific'] = #Organ_Specific
adata.obs['Dataset'] = #Dataset
adata.obs['InternDatasetNumber'] = #InternDatasetNumber
adata.obs['Dataset_status'] = #Dataset_status

adata.obs['celltype'] = adata.obs['celltype']
adata.obs['sub_celltype'] = adata.obs['sub_celltype']
adata.obs['Malignant'] = adata.obs['Malignant']

adata.obs['Patient'] = adata.obs['Patient']
adata.obs['Patient_Number'] = adata.obs['Patient_Number']
adata.obs['age'] = adata.obs['age']
adata.obs['sex'] = adata.obs['sex']
adata.obs['ethnicity'] = adata.obs['ethnicity']
adata.obs['health_status'] = #Organ

adata.obs['original_celltype_1'] = adata.obs['original_celltype_1']
adata.obs['original_celltype_2'] = adata.obs['original_celltype_2']
adata.obs['original_celltype_3'] = adata.obs['original_celltype_3']

In [ ]:
import scipy.sparse as sp
adata.X = sp.csr_matrix(adata.X)

In [ ]:
adata.obs_names_make_unique()

In [ ]:
path = 'adata/'

In [ ]:
adata.write(path + 'FileName.h5ad')